In [1]:
!nvidia-smi

Tue Mar 30 02:33:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install feedback-transformer-pytorch
!pip install transformers
!pip install torchtext

     |████████████████████████████████| 2.0MB 5.5MB/s 
     |████████████████████████████████| 890kB 35.8MB/s 
     |████████████████████████████████| 3.2MB 28.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=4dc99b0b9f2a497e72241aabd3ffdb201140cf8515b4b81aa7e3a491c8569851
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
%cd '/content/drive/MyDrive'
!ls

/content/drive/MyDrive
'Colab Notebooks'  'FeedBack Transformer'   Tokenizer_26_03
 Data		    GPT-2		    VnCoreNLP-master
 Đức		    PhoBERT_base_fairseq


In [5]:
import torch
from transformers import AutoModel, AutoTokenizer
from feedback_transformer_pytorch import FeedbackTransformer
from transformers import RobertaTokenizerFast
import os
import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from filelock import FileLock
from transformers.utils import logging
from typing import Dict, List, Optional
import pickle
import random
import time
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import numpy as np

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = RobertaTokenizerFast.from_pretrained("./Tokenizer_26_03", max_len=512)
tokenizer.add_tokens('\n')
vocab_size= tokenizer.vocab_size
vocab_size = vocab_size + 1

In [8]:
import torch
from feedback_transformer_pytorch import FeedbackTransformer

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

model = FeedbackTransformer(
    num_tokens = vocab_size,           # number of tokens
    dim = 512,                    # dimension
    depth = 6,                    # depth
    seq_len = 8,                  # the sequence length of each segment or window
    mem_len = 256,                # length of the memory buffer
    dim_head = 64,                # dimension of each head
    heads = 8,                    # number of heads
    attn_dropout = 0.1,           # attention dropout
    ff_dropout = 0.1              # feedforward dropout
).to(device)

In [9]:
lr_rate = 0.0001
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr_rate)
nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id).to(device)

CrossEntropyLoss()

In [22]:
def save_checkpoint(state, filename= "FeedBack Transformer/feedback_transformer_checkpoint_update.pth.tar"):
    print("Saving checkpoint")
    torch.save(state,filename)

def load_checkpoint(state):
    print("Load checkpoint")
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])


In [23]:
load_checkpoint(torch.load("FeedBack Transformer/feedback_transformer_checkpoint_update.pth.tar"))

Load checkpoint


In [24]:
class TextGenerator():

    def __init__(self, max_tokens, start_tokens, maxlen, model, tokenizer,device, topk):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.maxlen = maxlen
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.k = topk 

    def sample_from(self, logits):
        logits, indices = torch.topk(logits, k=self.k, sorted=True)
        return np.random.choice(indices.cpu().numpy())


    def gen_poem(self):
        start_tokens = [_ for _ in self.start_tokens]
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = self.maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:self.maxlen]
                sample_index = self.maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = torch.tensor([x], device= self.device)
            y = self.model(x)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
            # print(sample_token)
        output_token = [_ for _ in self.start_tokens + tokens_generated]
        poem = self.tokenizer.decode(output_token, skip_special_tokens=True)
        print(f"generated text:\n{poem}\n")

In [30]:
num_token_generated = 65
hint = 'quê hương là một tiếng ve'
start_tokens = tokenizer.encode(hint)[:-1]
generator = TextGenerator(max_tokens= num_token_generated, start_tokens= start_tokens, maxlen= 300, model= model, tokenizer= tokenizer,device= device, topk= 1)
generator.gen_poem()

generated text:
quê hương là một tiếng ve
vi vu tiếng sáo trưa hè về thôn
quê hương là những tiếng chim
nối tình với đất của miền xa xôi

quê hương là cả tình tôi
hương đồng gió nội đến từ ngày xưa
quê hương là tiếng thoi đưa
mùi trong tay mới sớm trưa ngọt ngào

quê hương là những khát khao

